# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * FastText
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import FastText, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
def Tagging(sentence, tagger, stopwords):
    pos = pd.DataFrame(tagger.pos(sentence))
    pos = pos[~pos[0].isin(stopwords)]
    pos = pos.apply(lambda x: x[0] +'/'+x[1], axis = 1)
    return pos.tolist()

In [4]:
@jit
def Tagging2(sentence, tagger, stopwords):
    pos = pd.Series(tagger.pos(sentence)).str[0]
    pos = pos[~pos.isin(stopwords)]
    return pos.tolist()

In [5]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [6]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [7]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [8]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'


## Category

In [9]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## TaggedDocument For FastText

In [10]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Comments Data

In [11]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDB = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDB, 'comments')

## FastText Model

### Twitter

In [12]:
taggerName = 'ct'

#### News to tagged Document

In [13]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

100%|██████████| 15120/15120 [30:23<00:00,  8.29it/s]


### Mecab

In [14]:
taggerName = 'mecab'

#### News to tagged Document

In [15]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

100%|██████████| 15120/15120 [05:41<00:00, 44.31it/s]


#### Load Model

In [ ]:
model1 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
model2 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
model3 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')

In [ ]:
name1 = '-'.join(re.split('[\(\),\/]',str(model1)))+taggerName
name2 = '-'.join(re.split('[\(\),\/]',str(model2)))+taggerName
name3 = '-'.join(re.split('[\(\),\/]',str(model3)))+taggerName

In [ ]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()

#### model1

In [ ]:
classifierList = glob(classifierPath+'*'+name1)

In [ ]:
x2 = list(filter(lambda y: x in y, classifierList))

In [ ]:
cls1 = pickle.load(open(x2[0], 'rb'))

# News

In [ ]:
site = 'Naver'
#site = 'daum'
for idx in naverData.index:
    data = naverData.loc[idx]
    text = data['title'] +'.\n' + data['mainText']
    tagged = TaggedDocument(Tagging(text, ct, stopwords), ['doc_'+str(idx)], [0.5])
    iv = model1.infer_vector(tagged.words)
    commentsForNews = useCollection.find({'site':site, 'category':data['category'], 'date':data['date'], 'rank':int(data['rank'])})
    print ('Number of comments for this news : {}'.format(commentsForNews.count()))
    for comment in commentsForNews:
        comm = comment['comments']
        comm_tagged = TaggedDocument(Tagging(comm, ct, stopwords), ['doc_'+str(idx)], [0.5])
        comm_iv = model1.infer_vector(comm_tagged.words)
        print (comm)
        break
    break

In [ ]:
cls1.predict(iv.reshape(1,-1))

* 주의 
> gensim의 doc2vec 모델의 infer_vector 함수를 이용해 벡터를 생성하면, 같은 단어열(문서)에 대해서도 여러번 호출하면 각기 다른 벡터가 나온다  
> 이는 문서 벡터 추론 과정에서 랜덤하게 초기화하는 과정이 있기 때문인데, 문제는 결과로 나오는 벡터의 차이가 꽤 크다.  

In [ ]:
def Cls(text, model, num):
    mecab = Mecab()
    def tokenize_pos2(doc):
        return ['/'.join(t) for t in mecab.pos(doc)]
    TaggedDocument = namedtuple('TaggedDocument', 'words tags')
    x2 = TaggedDocument(tokenize_pos2(text), 0.5)
    out = list()
    for idx in range(num):
        x = model.infer_vector(x2.words)
        cl = classifier.predict(x.reshape(1,-1))
        out.append(cl)
    out = list(itertools.chain.from_iterable(out))
    cls2 = int(round(sum(out)/len(out)))
    return cls2

In [ ]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))
#keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

In [ ]:
i=0
outputDict  =dict()
collection = useCollection.find({'site':site})
for data in collection:
    idis = data['_id']._ObjectId__id.hex()
    text = data['title']+'. '+data['mainText']    
    commentCollection = dh.Use_Collection(useDb, 'comments')
    comments = commentCollection.find({'site':site, 'category':data['category'], 'date':data['date'], 'rank':data['rank'] })
    commEstimation = list()
    for comment in comments:
        comm = comment['comments']
        commEstimation.append(Cls(comm, model, 30))
    cls = sum(commEstimation)/ len(commEstimation)    
    outputDict[idis] = tmpdict = dict()
    tmpdict['title'] = data['title']
    tmpdict['maintextestimation'] = Cls(text, model, 50)
    tmpdict['keywordsindaum'] = data['keywords']
    tmpdict['analyzedkeywords'] = keywordsDict[idis]
    tmpdict['number_crawledcomments'] = comments.count()
    tmpdict['commentsestimation'] = cls
    tmpdict['press'] = data['press']
    tmpdict['number_of_comment'] = data['number_of_comment']
pickle.dump(outputDict, open('./outcome/outcome_naver_by_mecab.pickled','wb'))

In [ ]:
pickle.dump(outputDict, open('./outcome/outcome_naver_by_mecab.pickled','wb'))